Import yearly cleanup script

In [1]:
import yearly_cleanup as yc

Input your cleanup year

In [2]:
year = yc.get_year()

Rounding up releases for: 2024



Retrieve all the subdirectories matching your cleanup year

In [3]:
yc.get_yearly_releases(year)

Found 13 release folders for year 2024:


['../site/releases/2024-aug-13',
 '../site/releases/2024-dec-06',
 '../site/releases/2024-dec-24',
 '../site/releases/2024-feb-14',
 '../site/releases/2024-jan-18',
 '../site/releases/2024-jan-26',
 '../site/releases/2024-jul-22',
 '../site/releases/2024-jun-10',
 '../site/releases/2024-mar-27',
 '../site/releases/2024-may-22',
 '../site/releases/2024-oct-22',
 '../site/releases/2024-sep-09',
 '../site/releases/2024-sep-25']

Create the new yearly folder

In [4]:
yearly_path = yc.create_year_folder(year)

Created folder: ../site/releases/2024/



Move the subdirectories matching your cleanup year to the new yearly folder

In [5]:
yc.move_yearly_releases(yearly_path, yc.release_folders)

Moved: '../site/releases/2024-aug-13' to '../site/releases/2024/2024-aug-13'
Moved: '../site/releases/2024-dec-06' to '../site/releases/2024/2024-dec-06'
Moved: '../site/releases/2024-dec-24' to '../site/releases/2024/2024-dec-24'
Moved: '../site/releases/2024-feb-14' to '../site/releases/2024/2024-feb-14'
Moved: '../site/releases/2024-jan-18' to '../site/releases/2024/2024-jan-18'
Moved: '../site/releases/2024-jan-26' to '../site/releases/2024/2024-jan-26'
Moved: '../site/releases/2024-jul-22' to '../site/releases/2024/2024-jul-22'
Moved: '../site/releases/2024-jun-10' to '../site/releases/2024/2024-jun-10'
Moved: '../site/releases/2024-mar-27' to '../site/releases/2024/2024-mar-27'
Moved: '../site/releases/2024-may-22' to '../site/releases/2024/2024-may-22'
Moved: '../site/releases/2024-oct-22' to '../site/releases/2024/2024-oct-22'
Moved: '../site/releases/2024-sep-09' to '../site/releases/2024/2024-sep-09'
Moved: '../site/releases/2024-sep-25' to '../site/releases/2024/2024-sep-25'

Copies the yearly release template into the new yearly folder

In [6]:
yearly_release = yc.copy_template(yearly_path, year)

Copied ../internal/templates/yearly-releases.qmd template to: '../site/releases/2024/2024-releases.qmd'


- Edit the title of the new yearly release page
- Edit the listing ID of the new yearly release page
- Change the new listing embed on the yearly release page

In [7]:
if yearly_release:
    yc.update_template(yearly_release, year)

Updated '../site/releases/2024/2024-releases.qmd' with the year 2024. Edited lines: [3, 9, 19].


Get the moved folders to add to the listing

In [8]:
release_listings = yc.get_release_listings(yearly_path)

Found 13 release notes in ../site/releases/2024/:
2024-dec-24/release-notes.qmd
2024-dec-06/release-notes.qmd
2024-oct-22/release-notes.qmd
2024-sep-25/release-notes.qmd
2024-sep-09/release-notes.qmd
2024-aug-13/release-notes.qmd
2024-jul-22/release-notes.qmd
2024-jun-10/release-notes.qmd
2024-may-22/release-notes.qmd
2024-mar-27/release-notes.qmd
2024-feb-14/release-notes.qmd
2024-jan-26/release-notes.qmd
2024-jan-18/release-notes.qmd


Add the new release files to the listing

In [9]:
if release_listings:
    yc.update_listing(yearly_release, release_listings)

Updated '../site/releases/2024/2024-releases.qmd' with release listings. Added lines: [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27].


Update _quarto.yml with new yearly sidebar folder

In [10]:
yc.update_quarto_yaml(year)

Modified lines: [173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185]
Added 2024 folder to the sidebar in _quarto.yml


Fix broken filepaths